# GCP Professional Data Engineer
### Serverless Data Analysis with Google BigQuery and Cloud Dataflow
#### Modules:
- Serverless Data Analysis with BigQuery
- Autoscaling Data Processing Pipelines with Dataflow

#### Learning Objectives:

- Build up a complex BigQuery using clauses, inner selects, built-in functions and joins
- Load and export data to/from BigQuery
- Identify need for nested, repeated fields and user-defined functions

## Module 1: Serverless Data Analysis with Big Query
#### Topics:
- Queries
- Functions
- Load & export data
- Nested, repeated fields
- Window functions
- User defined functions

### BigQuery Overview
#### BigQuery Benefits:
- Interactive analysis of petabyte scale databases
- Familiar SQL 2011 query language
- Nested and repeat fields, user defined functions in Javascript
- Data storage is inexpensive

#### BigQuery Sample Architecture
##### Project (billing, top-level container)
- Limit access to datasets and jobs
- Manage billing

##### Dataset (organization, access control)
- Access Control Lists for Reader/Writer/Owner
- Applied to all tables/views in dataset

##### Table (data w/ schema)
- Columnar storage
- Views are in virtual tables defined by SQL query
- Tables can be external (Cloud Storage, etc.)
- Each column is storage in a separated, encrypted file

##### Jobs (query, import, export, copy) 
- Repeated or long running action
- Can be cancelled

### Lab: Building a BigQuery Query
#### Objectives:
- Create and run a query
- Modify the query to add clauses, subqueries, built-in functions and joins.

#### Task 1: Create and Run a Query
- In the Console, on the Products & services menu () click BigQuery. Click on the Compose Query button on top left, and then click on Show Options, and ensure you are using Standard SQL. You are using Standard SQL if the Use Legacy SQL checkbox is unchecked.
- Click Hide Options.
- In the New Query window, type (or copy-and-paste) the following query:

In [ ]:
# sql
SELECT
  airline,
  date,
  departure_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_delay > 0
  AND departure_airport = 'LGA'
LIMIT
  100

#### Task 2: Aggregate and Boolean Fxns
- In the New Query window, type the following query(s):

In [ ]:
# total number of flights departed from LGA
SELECT
  airline,
  COUNT(departure_delay)
FROM
   `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_airport = 'LGA'
  AND date = '2008-05-13'
GROUP BY
  airline
ORDER BY airline

# total number of late flights from LGA
SELECT
  airline,
  COUNT(departure_delay)
FROM
   `bigquery-samples.airline_ontime_data.flights`
WHERE
  departure_delay > 0 AND
  departure_airport = 'LGA'
  AND date = '2008-05-13'
GROUP BY
  airline
ORDER BY airline

# total number of flights AND total delayed flights
SELECT
  f.airline,
  COUNT(f.departure_delay) AS total_flights,
  SUM(IF(f.departure_delay > 0, 1, 0)) AS num_delayed
FROM
   `bigquery-samples.airline_ontime_data.flights` AS f
WHERE
  f.departure_airport = 'LGA' AND f.date = '2008-05-13'
GROUP BY
  f.airline

#### Task 3: String Operations, Joins, & Subqueries

In [ ]:
SELECT
  CONCAT(CAST(year AS STRING), '-', LPAD(CAST(month AS STRING),2,'0'), '-', LPAD(CAST(day AS STRING),2,'0')) AS rainyday
FROM
  `bigquery-samples.weather_geo.gsod`
WHERE
  station_number = 725030
  AND total_precipitation > 0

# join weather data and flight information
SELECT
  f.airline,
  SUM(IF(f.arrival_delay > 0, 1, 0)) AS num_delayed,
  COUNT(f.arrival_delay) AS total_flights
FROM
  `bigquery-samples.airline_ontime_data.flights` AS f
JOIN (
  SELECT
    CONCAT(CAST(year AS STRING), '-', LPAD(CAST(month AS STRING),2,'0'), '-', LPAD(CAST(day AS STRING),2,'0')) AS rainyday
  FROM
    `bigquery-samples.weather_geo.gsod`
  WHERE
    station_number = 725030
    AND total_precipitation > 0) AS w
ON
  w.rainyday = f.date
WHERE f.arrival_airport = 'LGA'
GROUP BY f.airline

# fraction of flights delayed per airline
SELECT
  airline,
  num_delayed,
  total_flights,
  num_delayed / total_flights AS frac_delayed
FROM (
SELECT
  f.airline AS airline,
  SUM(IF(f.arrival_delay > 0, 1, 0)) AS num_delayed,
  COUNT(f.arrival_delay) AS total_flights
FROM
  `bigquery-samples.airline_ontime_data.flights` AS f
JOIN (
  SELECT
    CONCAT(CAST(year AS STRING), '-', LPAD(CAST(month AS STRING),2,'0'), '-', LPAD(CAST(day AS STRING),2,'0')) AS rainyday
  FROM
    `bigquery-samples.weather_geo.gsod`
  WHERE
    station_number = 725030
    AND total_precipitation > 0) AS w
ON
  w.rainyday = f.date
WHERE f.arrival_airport = 'LGA'
GROUP BY f.airline
  )
ORDER BY
  frac_delayed ASC

### End Lab
### Lab: Loading & Exporting Data
#### Objectives:
- Load a CSV file into a BigQuery table using the web UI
- Load a JSON file into a BigQuery table using the CLI
- Export a table using the web UI

#### Task 1: Upload the data using the web UI

- Return to the browser tab containing Console.
- In the Console, on the Products & services menu () click BigQuery.
- In the left column, beneath the text box, find your project name. To the right of the project name, click the blue arrow. - Choose Create new dataset.
- In the ‘Create Dataset' dialog, for Dataset ID, type cpb101_flight_data and click OK.
- Download the following file to your local machine. This file contains the data that will populate the first table.

[Download airports.csv](https://storage.googleapis.com/cloud-training/CPB200/BQ/lab4/airports.csv)

- Create a new table in the cpb101_flight_data dataset to store the data from the CSV file. Mouse over the line with the name of the dataset. This will reveal the create table icon, which looks like a plus sign. Click the create table icon (the plus sign) to the right of the cpb101_flight_data dataset.
- On the Create Table page, in the Source Data section:
    - For Location, leave File upload selected.
    - To the right of File upload, click Choose file, then browse to and select airports.csv.
    - Verify File format is set to CSV.
In the Destination Table section:
    - For Table name, leave cpb101_flight_data selected.
    - For Destination table name, type AIRPORTS.
    - For Table type, Native table should be selected and unchangeable.
- In the Schema section:
- Add fields one at a time. The airports.csv has the following fields: IATA, AIRPORT, CITY, STATE, COUNTRY which are of type STRING and LATITUDE, LONGITUDE which are of type FLOAT. Make all these fields REQUIRED.
- In the Options section:
    - For Field delimiter, verify Comma is selected.
    - Since airports.csv contains a single header row, for Header rows to skip, type 1.
    - Accept the remaining default values and click Create Table. BigQuery creates a load job to create the table and upload data into the table (this may take a few seconds). You can track job progress by clicking Job History
- Once the load job is complete, click cpb101_flight_data > AIRPORTS.
- On the Table Details page, click Details to view the table properties and then click Preview to view the table data.

#### Task 2. Upload the data using the CLI
- Return to the browser tab containing Cloud Shell.
- In Cloud Shell, enter the following command to download the schema file for the table to your working directory. (The file is schema_flight_performance.json)
``` 
curl https://storage.googleapis.com/cloud-training/CPB200/BQ/lab4/schema_flight_performance.json -o schema_flight_performance.json
```

- Next, you will create a table in the dataset using the schema file you downloaded to Cloud Shell and data from JSON files that are in Cloud Storage. The JSON files have URIs like the following:
```
gs://cloud-training/CPB200/BQ/lab4/domestic_2014_flights_*.json
```

##### Note that your Project ID is stored as a variable in Cloud Shell ($DEVSHELL_PROJECT_ID) so there's no need for you to remember it. If you require it, you can view your Project ID in the command line to the right of your username (after the @ symbol).

- In Cloud Shell, create a table named flights_2014 in the cpb101_flight_data dataset with this command:

``` 
bq load --source_format=NEWLINE_DELIMITED_JSON $DEVSHELL_PROJECT_ID:cpb101_flight_data.flights_2014 gs://cloud-training/CPB200/BQ/lab4/domestic_2014_flights_*.json ./schema_flight_performance.json 

```
- There are multiple JSON files in the Cloud Storage bucket. They are named according to the convention: domestic_2014_flights_*.json. The wildcard (*) character in the command is used to include all of the .json files in the bucket.
- Once the table is created, type the following command to verify table flights_2014 exists in dataset cpb101_flight_data:

```
bq ls $DEVSHELL_PROJECT_ID:cpb101_flight_data
```

#### Task 3: Export table
- In the Console, on the Products & services menu () click Home
- Select and copy the Project ID. For simplicity you will use the Qwiklabs Project ID, which is already globally unique, as the bucket name.
- In the Console, on the Products & services menu () click Storage > Browser.
- Click Create Bucket.
- Specify the following, and leave the remaining settings as their defaults:
- Click Create.
- Record the name of your bucket. You will need it in subsequent tasks.
- In Cloud Shell enter the following to create an environment variable named "BUCKET" and verify that it exists with the echo command.

```
BUCKET= |your unique bucket name (Project ID)|

echo $BUCKET
```

- You can use BUCKET in Cloud Shell commands. And if you need to enter the bucket name <your-bucket> in a text field in Console, you can quickly retrieve the name with "echo $BUCKET".
- Return to the BigQuery web UI. If it is not already open, open Console. On the Products & services menu () click BigQuery.
- Select the AIRPORTS table that you created recently, and using the "down" button to its right, select the option for Export Table.
- In the dialog, specify gs://<YOUR-BUCKET>/bq/airports.csv and click OK.
- Use the CLI to export the table:

```
bq extract cpb101_flight_data.AIRPORTS gs://$BUCKET/bq/airports2.csv
```

- In the Console, on the Products & services menu () click Storage > Browser. Browse to your bucket and ensure that both .csv files have been created.

### End Lab

### Lab: Advanced SQL Queries
#### Objectives: 
- Use Nested fields, Regular expressions, With statement, and Group and Having
- Extract programming information about code commits

#### Task 1: Get information about code commits
- In the Console, on the Products & services menu () click BigQuery.
- Compose a new query, making sure that the "Legacy SQL" option is not checked (you are using Standard SQL).
```
SELECT
  author.email,
  diff.new_path AS path,
  author.date
FROM
  `bigquery-public-data.github_repos.commits`,
  UNNEST(difference) diff
WHERE
  EXTRACT(YEAR
  FROM
    author.date)=2016
LIMIT 10
```
- Play a little with the query above to understand what it is doing. For example, instead of author.email, try just author. What type of field is author?
- Change diff.new_path to difference.new_path. Why does it not work? Replace difference.new_path by difference[OFFSET(0)].new_path. Does this work? Why? What is the UNNEST doing?

#### Task 2: Extract programming language
- Run the following query. Remember to uncheck Legacy SQL.

```
SELECT
  author.email,
  LOWER(REGEXP_EXTRACT(diff.new_path, r'\.([^\./\(~_ \- #]*)$')) lang,
  diff.new_path AS path,
  author.date
FROM
  `bigquery-public-data.github_repos.commits`,
  UNNEST(difference) diff
WHERE
  EXTRACT(YEAR
  FROM
    author.date)=2016
LIMIT
  10
```
- Modify the query above to only use lang if the language consists purely of letters and has a length that is fewer than 8 characters.
- Modify the query above to group by language and list in descending order of the number of commits.
- Here is one solution:
```
WITH
  commits AS (
  SELECT
    author.email,
    LOWER(REGEXP_EXTRACT(diff.new_path, r'\.([^\./\(~_ \- #]*)$')) lang,
    diff.new_path AS path,
    author.date
  FROM
    `bigquery-public-data.github_repos.commits`,
    UNNEST(difference) diff
  WHERE
    EXTRACT(YEAR
    FROM
      author.date)=2016 )
SELECT
  lang,
  COUNT(path) AS numcommits
FROM
  commits
WHERE
  LENGTH(lang)<8
  AND lang IS NOT NULL
  AND REGEXP_CONTAINS(lang, '[a-zA-Z]')
GROUP BY
  lang
HAVING
  numcommits > 100
ORDER BY
  numcommits DESC
```

#### Task 3: Weekend or weekday?
- Modify the previous query to extract the day of the week from author.date. Days 2 to 6 are weekdays.
```
WITH
  commits AS (
  SELECT
    author.email,
    EXTRACT(DAYOFWEEK
    FROM
      author.date) BETWEEN 2
    AND 6 is_weekday,
    LOWER(REGEXP_EXTRACT(diff.new_path, r'\.([^\./\(~_ \- #]*)$')) lang,
    diff.new_path AS path,
    author.date
  FROM
    `bigquery-public-data.github_repos.commits`,
    UNNEST(difference) diff
  WHERE
    EXTRACT(YEAR
    FROM
      author.date)=2016)
SELECT
  lang,
  is_weekday,
  COUNT(path) AS numcommits
FROM
  commits
WHERE
  lang IS NOT NULL
GROUP BY
  lang,
  is_weekday
HAVING
  numcommits > 100
ORDER BY
  numcommits DESC
```
#### Note: Ignoring file extensions that do not correspond to programming languages, it appears that the most popular weekend programming languages are JavaScript, PHP and C (*.h is the C header file), Java, and Python.

### End Lab

### Module 1 Review

1.) I want to query a table, then query within the results of that query. Which of these is the BEST way to do this?
- Use a subquery of the form SELECT ... FROM(SELECT) ...

2.) Which of the following statements are true?
- Dataflow transforms both batch and streaming pipelines
- Dataflow executes Apache Beam pipelines

Incorrect: 
- Side-inputs in Dataflow are a way to export data from one pipeline to share with another pipeline
- Map operations in MapReduce and be performed by Combine transforms in Dataflow

## Module 2: Autoscaling Data Processing Pipelines with Dataflow
#### Topics:
- Pipeline concepts
- MapReduce
- Side inputs
- Streaming